# 1 Why Synthetic Data Before Fine-Tuning

By now, the pattern should feel familiar: we don't escalate until the system tells us to.
Fine-tuning is no different. Before we change a model's weights, we need something to train on. And in most enterprise engagements, the training data simply does not exist in a usable form.

Customers have documents. They have institutional knowledge. They have years of accumulated expertise encoded in PDFs, wikis, and policy manuals. What they almost never have is a structured dataset of questions and high-quality answers that a model can learn from directly.

This is the gap that Synthetic Data Generation fills.

SDG is not about fabricating information. It is about transforming existing knowledge into a format that training pipelines can consume. We take the documents we have already ingested, chunked, and validated through retrieval, and we use them to generate question-answer pairs that reflect the domain. The documents contain the knowledge. SDG extracts the signal and reshapes it into something a model can actually learn from.

Why can't we just use the documents directly? Because training a model requires examples of the behavior you want. A 200-page rulebook is not an example of behavior. It is reference material. The model needs to see questions asked and answered correctly, repeatedly, across the full surface area of the domain. That is what SDG produces.
To do this in practice, we use sdg_hub, the Red Hat AI Innovation Team's open-source toolkit for building synthetic data generation pipelines. The framework is built around two core concepts: blocks and flows. Blocks are composable processing units, each responsible for a single transformation, such as generating a question from a document chunk, producing an answer, or evaluating faithfulness. Flows chain blocks together into complete pipelines defined in YAML. You describe the sequence of transformations declaratively, and the framework handles orchestration, validation, and execution.

In concrete terms, the workflow looks like this. You point a flow at your ingested documents. The flow discovers the dataset schema it needs, generates candidate question-answer pairs using a hosted LLM, and then runs built-in evaluation blocks that score each pair for faithfulness and relevancy. The output is a structured dataset of domain-specific Q&A pairs, each grounded in your source material and scored for quality. That dataset becomes the input for fine-tuning.

There is an important sequence dependency here. SDG quality is directly tied to everything we built earlier in this lab. The ingestion pipeline determines whether the source text is clean. The chunking strategy determines whether the generated pairs are coherent or fragmented. The retrieval layer determines whether we can validate that generated answers are actually grounded in real content. If any of those upstream stages are broken, SDG will faithfully reproduce the damage. It will generate confident, well-structured question-answer pairs that are subtly wrong, and you will carry that error forward into training.

This is why SDG comes after retrieval and evaluation, not before. It is not a shortcut. It is a manufacturing step that depends on the quality of every step that preceded it.

When the pipeline is sound, SDG gives us something powerful: enough structured, domain-specific training signal to make fine-tuning viable without requiring the customer to hand-label thousands of examples. That is a meaningful reduction in cost, time, and organizational friction.

But the order matters. Documents first. Ingestion second. Retrieval third. Evaluation fourth. And only then, when the system is stable and the failures are understood, do we generate the data that prepares us for model adaptation.
SDG is the bridge between "the system works well enough" and "the model needs to internalize this domain." It is not the destination. It is how we get there responsibly.

## 1.1 Install SDG Hub
Before we can generate anything, we need the toolkit. SDG Hub is a modular Python framework built by the Red Hat AI Innovation Team. It is open source, Apache 2.0 licensed, and designed specifically for building synthetic data generation pipelines using composable blocks and flows.

The core install pulls in the library itself. The `examples` extra adds the pre-built flows we will use in this section. These pre-built flows matter because they encode tested, validated generation pipelines that we can use out of the box rather than assembling one from scratch during a lab.

In [2]:
! pip install sdg-hub[examples] -q

Note: Earlier versions of the documentation reference a `[vllm]` extra. As of version 0.8.3, that extra has been removed. The core library already supports connecting to any OpenAI-compatible API endpoint, including vLLM, Ollama, and hosted services like the MaaS endpoint we configured in Section 2. No separate vLLM integration package is needed.
With the library installed, we can confirm that everything registered correctly by discovering the available flows and blocks. Flows are complete generation pipelines defined in YAML. Blocks are the individual processing units that flows chain together. Think of blocks as the atoms and flows as the molecules.

## 1.2 Import Libraries and Discover Available Flows

With the library installed, we can confirm that everything registered correctly by discovering the available flows and blocks. Flows are complete generation pipelines defined in YAML. Blocks are the individual processing units that flows chain together. Think of blocks as the atoms and flows as the molecules.

In [3]:
from sdg_hub.core.flow import FlowRegistry, Flow
from sdg_hub.core.blocks import BlockRegistry
from datasets import Dataset

# Auto-discover all registered flows and blocks
FlowRegistry.discover_flows()
BlockRegistry.discover_blocks()

# See what shipped with the install
print("Available flows:")
for name in FlowRegistry.list_flows():
    print(f"  - {name}")

[23:11:19] INFO     Discovered 7 flows                                                              ]8;id=735809;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/registry.py\registry.py]8;;\:]8;id=136869;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/registry.py#126\126]8;;\

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID               ┃ Name                  ┃ Author                ┃ Tags                  ┃ Description          ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ clean-shadow-397 │ Advanced Japanese     │ SDG Hub Contributors  │ question-generation,  │ A comprehensive flow │
│                  │ Document Grounded     │                       │ knowledge-extraction, │ that generates       │
│                  │ Question-Answer       │                       │ qa-pairs,             │ high-quality         │
│                  │ Generation Flow for   │                       │ document-processing,  │ question-answer      │
│                  │ Knowledge Tuning      │                       │ educational,          │ pairs from Japanese  │
│                  │                       │                       │ multilingual,         │ input documents      │
│                  │                       │                       │ japanese              │ using multiple LLM   │
│                  │                       │                       │                       │ blocks for question  │
│                  │                       │                       │                       │ generation, answer   │
│                  │                       │                       │                       │ synthesis, and       │
│                  │                       │                       │                       │ quality evaluation.  │
│ epic-jade-656    │ Extractive Summary    │ SDG Hub Contributors  │ knowledge-tuning,     │ Generate extractive  │
│                  │ Knowledge Tuning      │                       │ document-internaliza… │ summary from the     │
│                  │ Dataset Generation    │                       │ question-generation,  │ input document. Each │
│                  │ Flow                  │                       │ knowledge-extractive… │ document is first    │
│                  │                       │                       │ qa-pairs,             │ converted into list  │
│                  │                       │                       │ extractive-summaries  │ of knowledge         │
│                  │                       │                       │                       │ segments for         │
│                  │                       │                       │                       │ creating extractive  │
│                  │                       │                       │                       │ summary and then     │
│                  │                       │                       │                       │ annotated with       │
│                  │                       │                       │                       │ context,             │
│                  │                       │                       │                       │ relationship and     │
│                  │                       │                       │                       │ relevance. This is   │
│                  │                       │                       │                       │ then converted into  │
│                  │                       │                       │                       │ Question-Answer      │
│                  │                       │                       │                       │ pairs.               │
│ green-clay-812   │ Structured Text       │ SDG Hub Contributors  │ text-analysis,        │ Multi-step pipeline  │
│                  │ Insights Extraction   │                       │ summarization, nlp,   │ for extracting       │
│                  │ Flow                  │                       │ structured-output,    │ structured insights  │
│                  │                       │                       │ insights,             │ from text including  │
│                  │                       │            

                                                 Available Blocks                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Block Name                  ┃ Category  ┃ Description                                                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AgentBlock                  │ agent     │ Execute agent frameworks (Langflow, etc.) on DataFrame rows           │
│ AgentResponseExtractorBlock │ agent     │ Extracts text content from agent framework responses                  │
│ ColumnValueFilterBlock      │ filtering │ Filters datasets based on column values using various comparison      │
│                             │           │ operations                                                            │
│ LLMChatBlock                │ llm       │ Unified LLM chat block supporting 100+ providers via LiteLLM          │
│ LLMResponseExtractorBlock   │ llm       │ Extracts specified fields from LLM response objects                   │
│ PromptBuilderBlock          │ llm       │ Formats prompts into structured chat messages or plain text using     │
│                             │           │ Jinja templates                                                       │
│ RegexParserBlock            │ parsing   │ Parses text content using regex patterns                              │
│ TagParserBlock              │ parsing   │ Parses text content using start/end tags                              │
│ TextParserBlock             │ parsing   │ DEPRECATED: Use TagParserBlock or RegexParserBlock                    │
│ DuplicateColumnsBlock       │ transform │ Duplicates existing columns with new names according to a mapping     │
│                             │           │ specification                                                         │
│ IndexBasedMapperBlock       │ transform │ Maps values from source columns to output columns based on choice     │
│                             │           │ columns using shared mapping                                          │
│ JSONStructureBlock          │ transform │ Combines multiple columns into a single column containing a           │
│                             │           │ structured JSON object                                                │
│ MeltColumnsBlock            │ transform │ Transforms wide dataset format into long format by melting columns    │
│                             │           │ into rows                                                             │
│ RenameColumnsBlock          │ transform │ Renames columns in a dataset according to a mapping specification     │
│ RowMultiplierBlock          │ transform │ Duplicates each row in the dataset a configurable number of times     │
│ SamplerBlock                │ transform │ Randomly samples n values from a list column and outputs to a new     │
│                             │           │ column                                                                │
│ TextConcatBlock             │ transform │ Combines multiple columns into a single column using a specified      │
│                             │           │ separator                                                             │
│ UniformColumnValueSetter    │ transform │ Replaces all values in a column with a single summary statistic       │
│                             │           │ (e.g., mode, mean, median)                                            │
└─────────────────────────────┴───────────┴───────────────────────────────────────────────────────────────────────┘

Summary: 18 blocks across 5 categories

Available flows:
  - {'id': 'loud-dawn-245', 'name': 'RAG Evaluation Dataset Flow'}
  - {'id': 'mild-thunder-748', 'name': 'Detailed Summary Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'stellar-peak-605', 'name': 'Document Based Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'epic-jade-656', 'name': 'Extractive Summary Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'heavy-heart-77', 'name': 'Key Facts Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'clean-shadow-397', 'name': 'Advanced Japanese Document Grounded Question-Answer Generation Flow for Knowledge Tuning'}
  - {'id': 'green-clay-812', 'name': 'Structured Text Insights Extraction Flow'}


You should see several pre-built flows in the output, including flows for question-answer generation, knowledge tuning, and reasoning data. We will be using one of these in the next step.

If the output is empty or the import fails, check that the install completed without errors and that your Python environment is 3.10 or newer.

## 1.3 Load a Pre-Built Q&A Generation Flow
Now that we can see what is available, we select the flow we will use for the rest of this section. We are using one of the pre-built flows that generates question-answer pairs from documents. This is the same pattern a customer would follow: take ingested documents, produce structured training data.



In [13]:
flow_name = "Advanced Document Grounded Question-Answer Generation Flow for Knowledge Tuning"
flow_path = FlowRegistry.get_flow_path(flow_name)
flow = Flow.from_yaml(flow_path)


/tmp/ipykernel_6538/862295401.py:2: DeprecationWarning: DeprecationWarning: The InstructLab Multi-Summary QA flow has been deprecated and removed. Consider using 'enhanced_summary_knowledge_tuning' instead.
  flow_path = FlowRegistry.get_flow_path(flow_name)


[03:27:39] WARNING  DeprecationWarning: The InstructLab Multi-Summary QA flow has been deprecated   ]8;id=15047;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/registry.py\registry.py]8;;\:]8;id=346654;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/registry.py#177\177]8;;\
                    and removed. Consider using 'enhanced_summary_knowledge_tuning' instead.                       

FlowValidationError: Flow path cannot be None. Please provide a valid YAML file path or check that the flow exists in the registry.

In [14]:
FlowRegistry.discover_flows()
flows = FlowRegistry.list_flows()
for f in flows:
    print(f)

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID               ┃ Name                  ┃ Author                ┃ Tags                  ┃ Description          ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ clean-shadow-397 │ Advanced Japanese     │ SDG Hub Contributors  │ question-generation,  │ A comprehensive flow │
│                  │ Document Grounded     │                       │ knowledge-extraction, │ that generates       │
│                  │ Question-Answer       │                       │ qa-pairs,             │ high-quality         │
│                  │ Generation Flow for   │                       │ document-processing,  │ question-answer      │
│                  │ Knowledge Tuning      │                       │ educational,          │ pairs from Japanese  │
│                  │                       │                       │ multilingual,         │ input documents      │
│                  │                       │                       │ japanese              │ using multiple LLM   │
│                  │                       │                       │                       │ blocks for question  │
│                  │                       │                       │                       │ generation, answer   │
│                  │                       │                       │                       │ synthesis, and       │
│                  │                       │                       │                       │ quality evaluation.  │
│ epic-jade-656    │ Extractive Summary    │ SDG Hub Contributors  │ knowledge-tuning,     │ Generate extractive  │
│                  │ Knowledge Tuning      │                       │ document-internaliza… │ summary from the     │
│                  │ Dataset Generation    │                       │ question-generation,  │ input document. Each │
│                  │ Flow                  │                       │ knowledge-extractive… │ document is first    │
│                  │                       │                       │ qa-pairs,             │ converted into list  │
│                  │                       │                       │ extractive-summaries  │ of knowledge         │
│                  │                       │                       │                       │ segments for         │
│                  │                       │                       │                       │ creating extractive  │
│                  │                       │                       │                       │ summary and then     │
│                  │                       │                       │                       │ annotated with       │
│                  │                       │                       │                       │ context,             │
│                  │                       │                       │                       │ relationship and     │
│                  │                       │                       │                       │ relevance. This is   │
│                  │                       │                       │                       │ then converted into  │
│                  │                       │                       │                       │ Question-Answer      │
│                  │                       │                       │                       │ pairs.               │
│ green-clay-812   │ Structured Text       │ SDG Hub Contributors  │ text-analysis,        │ Multi-step pipeline  │
│                  │ Insights Extraction   │                       │ summarization, nlp,   │ for extracting       │
│                  │ Flow                  │                       │ structured-output,    │ structured insights  │
│                  │                       │                       │ insights,             │ from text including  │
│                  │                       │            

{'id': 'loud-dawn-245', 'name': 'RAG Evaluation Dataset Flow'}
{'id': 'mild-thunder-748', 'name': 'Detailed Summary Knowledge Tuning Dataset Generation Flow'}
{'id': 'stellar-peak-605', 'name': 'Document Based Knowledge Tuning Dataset Generation Flow'}
{'id': 'epic-jade-656', 'name': 'Extractive Summary Knowledge Tuning Dataset Generation Flow'}
{'id': 'heavy-heart-77', 'name': 'Key Facts Knowledge Tuning Dataset Generation Flow'}
{'id': 'clean-shadow-397', 'name': 'Advanced Japanese Document Grounded Question-Answer Generation Flow for Knowledge Tuning'}
{'id': 'green-clay-812', 'name': 'Structured Text Insights Extraction Flow'}


In [9]:
qa_flows = FlowRegistry.search_flows(tag="question-generation")
print(qa_flows)

[{'id': 'mild-thunder-748', 'name': 'Detailed Summary Knowledge Tuning Dataset Generation Flow'}, {'id': 'stellar-peak-605', 'name': 'Document Based Knowledge Tuning Dataset Generation Flow'}, {'id': 'epic-jade-656', 'name': 'Extractive Summary Knowledge Tuning Dataset Generation Flow'}, {'id': 'heavy-heart-77', 'name': 'Key Facts Knowledge Tuning Dataset Generation Flow'}, {'id': 'clean-shadow-397', 'name': 'Advanced Japanese Document Grounded Question-Answer Generation Flow for Knowledge Tuning'}]


In [11]:
flow_name = "Document Based Knowledge Tuning Dataset Generation Flow"
flow_path = FlowRegistry.get_flow_path(flow_name)
flow = Flow.from_yaml(flow_path)


[23:20:42] INFO     Loading flow from:                                                          ]8;id=831906;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/serialization.py\serialization.py]8;;\:]8;id=131281;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/serialization.py#60\60]8;;\
                    /opt/app-root/lib64/python3.12/site-packages/sdg_hub/flows/qa_generation/do                    
                    cument_grounded_qa/enhanced_multi_summary_qa/doc_direct_qa/flow.yaml                           

In [15]:
default_model = flow.get_default_model()
recommendations = flow.get_model_recommendations()
print(f"Default model: {default_model}")
print(f"Recommendations: {recommendations}")

Default model: openai/gpt-oss-120b
Recommendations: {'default': 'openai/gpt-oss-120b', 'compatible': ['meta-llama/Llama-3.3-70B-Instruct', 'microsoft/phi-4', 'mistralai/Mixtral-8x7B-Instruct-v0.1'], 'experimental': []}


In [17]:
from sdg_hub.core.flow import FlowRegistry, Flow
from sdg_hub.core.blocks import BlockRegistry
from datasets import Dataset

# Auto-discover all registered flows and blocks
#FlowRegistry.discover_flows()
#BlockRegistry.discover_blocks()

# See what shipped with the install
print("Available flows:")
for name in FlowRegistry.list_flows():
    print(f"  - {name}")

Available flows:
  - {'id': 'loud-dawn-245', 'name': 'RAG Evaluation Dataset Flow'}
  - {'id': 'mild-thunder-748', 'name': 'Detailed Summary Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'stellar-peak-605', 'name': 'Document Based Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'epic-jade-656', 'name': 'Extractive Summary Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'heavy-heart-77', 'name': 'Key Facts Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'clean-shadow-397', 'name': 'Advanced Japanese Document Grounded Question-Answer Generation Flow for Knowledge Tuning'}
  - {'id': 'green-clay-812', 'name': 'Structured Text Insights Extraction Flow'}


In [18]:
flow_name = "Document Based Knowledge Tuning Dataset Generation Flow"
flow_path = FlowRegistry.get_flow_path(flow_name)
flow = Flow.from_yaml(flow_path)

[03:31:38] INFO     Loading flow from:                                                          ]8;id=365047;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/serialization.py\serialization.py]8;;\:]8;id=29685;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/serialization.py#60\60]8;;\
                    /opt/app-root/lib64/python3.12/site-packages/sdg_hub/flows/qa_generation/do                    
                    cument_grounded_qa/enhanced_multi_summary_qa/doc_direct_qa/flow.yaml                           

## 1.4 Configure the Model Backend

SDG Hub does not host or download models. It connects to an external endpoint that serves the LLM. This is the same architectural pattern we have been working with all lab: the model runs somewhere else, and our code talks to it over an API.

Let's first see what the flow recommends.

In [19]:
default_model = flow.get_default_model()
recommendations = flow.get_model_recommendations()
print(f"Default model: {default_model}")
print(f"Recommendations: {recommendations}")

Default model: openai/gpt-oss-120b
Recommendations: {'default': 'openai/gpt-oss-120b', 'compatible': ['meta-llama/Llama-3.3-70B-Instruct', 'microsoft/phi-4', 'mistralai/Mixtral-8x7B-Instruct-v0.1'], 'experimental': []}


### Try MaaS

In [22]:
API_KEY="sk-UFHcLOTZk_73o6YwVQhSiQ"
ENDPOINT_BASE="https://litellm-prod.apps.maas.redhatworkshops.io/v1"

In [23]:
flow.set_model_config(
    model="microsoft-phi-4",
    api_base=ENDPOINT_BASE,
    api_key=API_KEY,
)

[03:35:19] INFO     Auto-detected 3 LLM blocks for configuration: ['answer_generation',         ]8;id=394830;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py\model_config.py]8;;\:]8;id=210192;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py#242\242]8;;\
                    'eval_faithful_llm_chat', 'question_generation']                                               

           INFO     Successfully configured 3 LLM blocks with: model: 'microsoft-phi-4',        ]8;id=29182;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py\model_config.py]8;;\:]8;id=975196;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py#300\300]8;;\
                    api_base: 'https://litellm-prod.apps.maas.redhatworkshops.io/v1', api_key:                     
                    (redacted)                                                                                     

           INFO     Configured blocks: ['answer_generation', 'eval_faithful_llm_chat',          ]8;id=809180;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py\model_config.py]8;;\:]8;id=237504;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py#303\303]8;;\
                    'question_generation']                                                                         

## 1.5 Discover the Dataset Schema

In [26]:
schema_dataset = flow.get_dataset_schema()
print(f"Required columns: {list(schema_dataset.columns)}")
print(f"Schema:\n{schema_dataset.dtypes}")

Required columns: ['document', 'document_outline', 'domain', 'icl_document', 'icl_query_1', 'icl_query_2', 'icl_query_3']
Schema:
document            object
document_outline    object
domain              object
icl_document        object
icl_query_1         object
icl_query_2         object
icl_query_3         object
dtype: object


In [27]:
requirements = flow.get_dataset_requirements()
print(type(requirements))
print(requirements)

<class 'sdg_hub.core.flow.metadata.DatasetRequirements'>
required_columns=['document', 'document_outline', 'domain', 'icl_document', 'icl_query_1', 'icl_query_2', 'icl_query_3'] optional_columns=[] min_samples=1 max_samples=None column_types={} description='Input dataset should contain documents with text content and domain classification. Each document should be substantial enough for meaningful question generation (minimum 100 words recommended). The flow generates three types of summaries: detailed (n=20), extractive (n=10), and key facts (n=50), each producing corresponding QA pairs designed to help LLMs internalize document knowledge for knowledge tuning.'


## 1.6 Build the Seed Dataset

In [29]:
import nest_asyncio
nest_asyncio.apply()

In [31]:
# Load the Docling output from Section 3
with open("Basic-Fantasy-RPG-Rules-r142.md", "r", encoding="utf-8") as f:
    full_text = f.read()

# Use a meaningful section of the document as the source
# In production, you would iterate across many chunks or sections
document_chunk = full_text[3000:6000]

In [32]:
print(f"Document chunk length: {len(document_chunk)} characters")
print(f"\nFirst 300 characters:\n{document_chunk[:300]}...")

Document chunk length: 3000 characters

First 300 characters:
   Schoonover, Jason   Brentlinger, Chris Wolfmeyer, Josh   Eaton, Audra Brentlinger, Tim McAfee, Ike Borden, Cody  Drebenstedt, Joseph  BierFauble, Emily Drebenstedt, John Lopez, Pedro  Pablo  Miron  Pozo, Robert Odom, Sergio   I.    Nemirovsky, Will    E.    Sanders, Brian Scalise, Timothy  J.    ...


In [33]:
import pandas as pd

dataset = pd.DataFrame({
    'document': [document_chunk],
    'document_outline': [
        '1. Character Abilities and Ability Scores; '
        '2. Hit Points and Hit Dice; '
        '3. Character Classes and Prime Requisites'
    ],
    'domain': ['Tabletop RPG Rules'],
    'icl_document': [
        'Open Locks allows the Thief to unlock a lock without a proper key. '
        'It may only be tried once per lock. If the attempt fails, the Thief '
        'must wait until they have gained another level of experience before '
        'trying again.'
    ],
    'icl_query_1': ['What happens if a Thief fails an Open Locks attempt?'],
    'icl_query_2': ['Can a Thief retry a failed Open Locks check immediately?'],
    'icl_query_3': ['Does the Open Locks ability require a key?'],
})

print(f"Dataset shape: {dataset.shape}")
print(f"Columns: {list(dataset.columns)}")

Dataset shape: (1, 7)
Columns: ['document', 'document_outline', 'domain', 'icl_document', 'icl_query_1', 'icl_query_2', 'icl_query_3']


## 1.7 Dry Run

In [38]:
flow.set_model_config(
    model="openai/microsoft-phi-4",
    api_base=ENDPOINT_BASE,
    api_key=API_KEY,
)


print("Running dry run...")
dry_result = flow.dry_run(dataset, sample_size=1)

[03:42:36] INFO     Auto-detected 3 LLM blocks for configuration: ['answer_generation',         ]8;id=938049;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py\model_config.py]8;;\:]8;id=39629;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py#242\242]8;;\
                    'eval_faithful_llm_chat', 'question_generation']                                               

           INFO     Successfully configured 3 LLM blocks with: model: 'openai/microsoft-phi-4', ]8;id=85929;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py\model_config.py]8;;\:]8;id=94410;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py#300\300]8;;\
                    api_base: 'https://litellm-prod.apps.maas.redhatworkshops.io/v1', api_key:                     
                    (redacted)                                                                                     

           INFO     Configured blocks: ['answer_generation', 'eval_faithful_llm_chat',          ]8;id=548818;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py\model_config.py]8;;\:]8;id=658842;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/model_config.py#303\303]8;;\
                    'question_generation']                                                                         

Running dry run...


[03:42:36] INFO     Starting dry run for flow 'Document Based Knowledge Tuning Dataset Generation  ]8;id=51578;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=253615;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#614\614]8;;\
                    Flow' with 1 samples                                                                           

...

           INFO     Dry run executing block 14/14: eval_faithful_filter (ColumnValueFilterBlock)   ]8;id=306823;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=302540;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#647\647]8;;\

╭───────────────────────────────────────────── eval_faithful_filter ──────────────────────────────────────────────╮
│ 📊 Processing Input Data                                                                                        │
│ Block Type: ColumnValueFilterBlock                                                                              │
│ Input Rows: 8                                                                                                   │
│ Input Columns: 21                                                                                               │
│ Column Names: document, document_outline, domain, icl_document, icl_query_1, icl_query_2, icl_query_3,          │
│ base_document, question_generation_prompt, question_list, extract_questions_content, question,                  │
│ answer_generation_prompt, response_dict, extract_answer_content, response, eval_faithful_prompt,                │
│ eval_faithful_response_dict, extract_eval_faithful_content, faithfulness_explanation, faithfulness_judgment     │
│ Expected Output Columns: None specified                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── eval_faithful_filter - Complete ────────────────────────────────────────╮
│ ✅ Processing Complete                                                                                          │
│ Rows: 8 → 6                                                                                                     │
│ Columns: 21 → 21                                                                                                │
│ 📋 Final Columns: answer_generation_prompt, base_document, document, document_outline, domain,                  │
│ eval_faithful_prompt, eval_faithful_response_dict, extract_answer_content, extract_eval_faithful_content,       │
│ extract_questions_content, faithfulness_explanation, faithfulness_judgment, icl_document, icl_query_1,          │
│ icl_query_2, icl_query_3, question, question_generation_prompt, question_list, response, response_dict          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

           INFO     Dry run block 'eval_faithful_filter' completed: 6 samples, 21 columns, 0.01s   ]8;id=202417;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=695568;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#679\679]8;;\

           INFO     Dry run completed successfully for flow 'Document Based Knowledge Tuning       ]8;id=847275;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=568311;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#698\698]8;;\
                    Dataset Generation Flow' in 15.70s                                                             

In [39]:
print(f"Dry run completed in {dry_result['execution_time_seconds']:.2f}s")
print(f"Output columns: {list(dry_result['final_dataset']['columns'])}")

Dry run completed in 15.70s
Output columns: ['document', 'document_outline', 'domain', 'icl_document', 'icl_query_1', 'icl_query_2', 'icl_query_3', 'base_document', 'question_generation_prompt', 'question_list', 'extract_questions_content', 'question', 'answer_generation_prompt', 'response_dict', 'extract_answer_content', 'response', 'eval_faithful_prompt', 'eval_faithful_response_dict', 'extract_eval_faithful_content', 'faithfulness_explanation', 'faithfulness_judgment']


## 1.8 Generate Synthetic Q&A Pairs

In [41]:
import time

print("Generating synthetic data...")
start_time = time.time()
result = flow.generate(dataset)
elapsed = time.time() - start_time

print(f"Generation complete in {elapsed:.1f}s")
print(f"Generated {len(result)} QA pairs")

Generating synthetic data...


[03:45:27] INFO     Starting flow 'Document Based Knowledge Tuning Dataset Generation Flow' v2.0.0 ]8;id=483037;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=621557;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#448\448]8;;\
                    with 1 samples across 14 blocks                                                                

           INFO     Executing block 1/14: duplicate_document_col (DuplicateColumnsBlock)           ]8;id=871404;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=334114;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#220\220]8;;\

╭──────────────────────────────────────────── duplicate_document_col ─────────────────────────────────────────────╮
│ 📊 Processing Input Data                                                                                        │
│ Block Type: DuplicateColumnsBlock                                                                               │
│ Input Rows: 1                                                                                                   │
│ Input Columns: 7                                                                                                │
│ Column Names: document, document_outline, domain, icl_document, icl_query_1, icl_query_2, icl_query_3           │
│ Expected Output Columns: base_document                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────── duplicate_document_col - Complete ───────────────────────────────────────╮
│ ✅ Processing Complete                                                                                          │
│ Rows: 1 → 1                                                                                                     │
│ Columns: 7 → 8                                                                                                  │
│ 🟢 Added: base_document                                                                                         │
│ 📋 Final Columns: base_document, document, document_outline, domain, icl_document, icl_query_1, icl_query_2,    │
│ icl_query_3                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────── eval_faithful_filter - Complete ────────────────────────────────────────╮
│ ✅ Processing Complete                                                                                          │
│ Rows: 6 → 2                                                                                                     │
│ Columns: 21 → 21                                                                                                │
│ 📋 Final Columns: answer_generation_prompt, base_document, document, document_outline, domain,                  │
│ eval_faithful_prompt, eval_faithful_response_dict, extract_answer_content, extract_eval_faithful_content,       │
│ extract_questions_content, faithfulness_explanation, faithfulness_judgment, icl_document, icl_query_1,          │
│ icl_query_2, icl_query_3, question, question_generation_prompt, question_list, response, response_dict          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

           INFO     Block 'eval_faithful_filter' completed successfully: 2 samples, 21 columns     ]8;id=734434;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=853231;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#266\266]8;;\

╭────────────────────── Document Based Knowledge Tuning Dataset Generation Flow - Complete ───────────────────────╮
│                                        Flow Execution Summary                                                   │
│ ┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓           │
│ ┃ Block Name           ┃ Type            ┃   Duration ┃     Rows     ┃     Columns     ┃   Status   ┃           │
│ ┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩           │
│ │ duplicate_document_… │ DuplicateColum… │      0.00s │    1 → 1     │       +1        │     ✓      │           │
│ │ question_generation… │ PromptBuilderB… │      0.00s │    1 → 1     │       +1        │     ✓      │           │
│ │ question_generation  │ LLMChatBlock    │      3.43s │    1 → 1     │       +1        │     ✓      │           │
│ │ extract_questions    │ LLMResponseExt… │      0.00s │    1 → 1     │       +1        │     ✓      │           │
│ │ parse_question_list  │ TagParserBlock  │      0.00s │    1 → 6     │       +1        │     ✓      │           │
│ │ answer_generation_p… │ PromptBuilderB… │      0.00s │    6 → 6     │       +1        │     ✓      │           │
│ │ answer_generation    │ LLMChatBlock    │      2.99s │    6 → 6     │       +1        │     ✓      │           │
│ │ extract_answer       │ LLMResponseExt… │      0.00s │    6 → 6     │       +1        │     ✓      │           │
│ │ parse_response_dict  │ TagParserBlock  │      0.00s │    6 → 6     │       +1        │     ✓      │           │
│ │ eval_faithful_prompt │ PromptBuilderB… │      0.01s │    6 → 6     │       +1        │     ✓      │           │
│ │ eval_faithful_llm_c… │ LLMChatBlock    │      3.54s │    6 → 6     │       +1        │     ✓      │           │
│ │ extract_eval_faithf… │ LLMResponseExt… │      0.00s │    6 → 6     │       +1        │     ✓      │           │
│ │ parse_eval_faithful  │ TagParserBlock  │      0.00s │    6 → 6     │       +2        │     ✓      │           │
│ │ eval_faithful_filter │ ColumnValueFil… │      0.00s │    6 → 2     │        —        │     ✓      │           │
│ ├──────────────────────┼─────────────────┼────────────┼──────────────┼─────────────────┼────────────┤           │
│ │ TOTAL                │ 14 blocks       │     10.01s │   2 final    │    21 final     │   14/14    │           │
│ └──────────────────────┴─────────────────┴────────────┴──────────────┴─────────────────┴────────────┘           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

           INFO     Flow 'Document Based Knowledge Tuning Dataset Generation Flow' completed       ]8;id=833060;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=959783;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#548\548]8;;\
                    successfully: 2 final samples, 21 final columns                                                

Generation complete in 10.1s
Generated 2 QA pairs


In [42]:
elapsed

10.072309494018555

In [43]:
print(f"Generation complete in {elapsed:.1f}s")
print(f"Generated {len(result)} QA pairs")

Generation complete in 10.1s
Generated 2 QA pairs


In [44]:
for i in range(len(result)):
    print(f"QA Pair #{i + 1}")
    print(f"  Question:     {result['question'].iloc[i]}")
    print(f"  Answer:       {result['response'].iloc[i]}")
    if 'faithfulness_judgment' in result.columns:
        print(f"  Faithfulness: {result['faithfulness_judgment'].iloc[i]}")
    if 'relevancy_score' in result.columns:
        print(f"  Relevancy:    {result['relevancy_score'].iloc[i]}")
    print()

QA Pair #1
  Question:     Based on the document, what is the difference between 'Attacking From Behind' and a standard 'How to Attack' strategy?
  Answer:       The document you provided lists various sections from a text related to role-playing games, specifically mentioning topics such as "Attacking From Behind" and "How to Attack". However, it does not provide specific descriptions or details about these strategies themselves. Thus, based on the information available directly in the document, there is no explicit explanation of what distinguishes "Attacking From Behind" from a standard "How to Attack" strategy.

In general terms, in role-playing games, "Attacking From Behind" often involves specific bonuses or conditions that differ from a standard attack, as many systems provide benefits for hitting an opponent from a tactical advantage such as reduced defense or a surprise element. In contrast, a standard "How to Attack" strategy would discuss the general mechanics of engaging in

## 1.9 Iteration: Better Input, Better Output


In [45]:
print("Previous chunk (first 500 chars):")
print(document_chunk[:500])

Previous chunk (first 500 chars):
   Schoonover, Jason   Brentlinger, Chris Wolfmeyer, Josh   Eaton, Audra Brentlinger, Tim McAfee, Ike Borden, Cody  Drebenstedt, Joseph  BierFauble, Emily Drebenstedt, John Lopez, Pedro  Pablo  Miron  Pozo, Robert Odom, Sergio   I.    Nemirovsky, Will    E.    Sanders, Brian Scalise, Timothy  J.    Kuhn, and    Jeanne Mayer Mitchell

<!-- image -->

## TABLE OF CONTENTS

| PART 1: INTRODUCTION........................................1                                                               


In [46]:
# Find the section on Thief abilities - we know this has real rule content
search_term = "Open Locks"
idx = full_text.find(search_term)

if idx != -1:
    # Back up to capture context before the match, forward to capture the full section
    start = max(0, idx - 500)
    end = min(len(full_text), idx + 2500)
    better_chunk = full_text[start:end]
    print(f"Found '{search_term}' at position {idx}")
    print(f"Chunk range: {start} to {end} ({len(better_chunk)} characters)")
    print(f"\nFirst 500 characters:\n{better_chunk[:500]}...")
else:
    print(f"'{search_term}' not found in document")

Found 'Open Locks' at position 57479
Chunk range: 56979 to 59979 (3000 characters)

First 500 characters:
se abilities, as determined by the GM. The GM may choose to make any of these rolls on behalf of the player to help maintain the proper state of uncertainty. Also   note   that   the   GM   may   apply situational   adjustments   (plus   or   minus   percentage points) as they see fit; for instance, it's obviously harder to climb a wall slick with slime than one that is dry, so the GM might apply a penalty of 20% for the slimy wall.

## BASIC FANTASY RPG

## Thief Abilities

|   Thief Level |   ...


In [47]:
dataset_v2 = pd.DataFrame({
    'document': [better_chunk],
    'document_outline': [
        '1. Thief Special Abilities; '
        '2. Open Locks, Pick Pockets, and Remove Traps; '
        '3. Class Restrictions and Level Progression'
    ],
    'domain': ['Tabletop RPG Rules'],
    'icl_document': [
        'Open Locks allows the Thief to unlock a lock without a proper key. '
        'It may only be tried once per lock. If the attempt fails, the Thief '
        'must wait until they have gained another level of experience before '
        'trying again.'
    ],
    'icl_query_1': ['What happens if a Thief fails an Open Locks attempt?'],
    'icl_query_2': ['Can a Thief retry a failed Open Locks check immediately?'],
    'icl_query_3': ['Does the Open Locks ability require a key?'],
})

Notice what changed and what did not. The ICL queries are the same. The domain is the same. The flow configuration is the same. The model is the same. The only thing we changed is the document chunk. If the output improves, we know exactly why.

Run it.

In [48]:
import time

print("Generating with improved chunk...")
start_time = time.time()
result_v2 = flow.generate(dataset_v2)
elapsed = time.time() - start_time

print(f"Generation complete in {elapsed:.1f}s")
print(f"Generated {len(result_v2)} QA pairs")

Generating with improved chunk...


[03:51:40] INFO     Starting flow 'Document Based Knowledge Tuning Dataset Generation Flow' v2.0.0 ]8;id=216419;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=354842;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#448\448]8;;\
                    with 1 samples across 14 blocks                                                                

           INFO     Executing block 1/14: duplicate_document_col (DuplicateColumnsBlock)           ]8;id=130978;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=187014;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#220\220]8;;\

╭──────────────────────────────────────────── duplicate_document_col ─────────────────────────────────────────────╮
│ 📊 Processing Input Data                                                                                        │
│ Block Type: DuplicateColumnsBlock                                                                               │
│ Input Rows: 1                                                                                                   │
│ Input Columns: 7                                                                                                │
│ Column Names: document, document_outline, domain, icl_document, icl_query_1, icl_query_2, icl_query_3           │
│ Expected Output Columns: base_document                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────── duplicate_document_col - Complete ───────────────────────────────────────╮
│ ✅ Processing Complete                                                                                          │
│ Rows: 1 → 1                                                                                                     │
│ Columns: 7 → 8                                                                                                  │
│ 🟢 Added: base_document                                                                                         │
│ 📋 Final Columns: base_document, document, document_outline, domain, icl_document, icl_query_1, icl_query_2,    │
│ icl_query_3                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────── eval_faithful_filter - Complete ────────────────────────────────────────╮
│ ✅ Processing Complete                                                                                          │
│ Rows: 7 → 6                                                                                                     │
│ Columns: 21 → 21                                                                                                │
│ 📋 Final Columns: answer_generation_prompt, base_document, document, document_outline, domain,                  │
│ eval_faithful_prompt, eval_faithful_response_dict, extract_answer_content, extract_eval_faithful_content,       │
│ extract_questions_content, faithfulness_explanation, faithfulness_judgment, icl_document, icl_query_1,          │
│ icl_query_2, icl_query_3, question, question_generation_prompt, question_list, response, response_dict          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

           INFO     Block 'eval_faithful_filter' completed successfully: 6 samples, 21 columns     ]8;id=706950;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=40023;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#266\266]8;;\

╭────────────────────── Document Based Knowledge Tuning Dataset Generation Flow - Complete ───────────────────────╮
│                                        Flow Execution Summary                                                   │
│ ┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓           │
│ ┃ Block Name           ┃ Type            ┃   Duration ┃     Rows     ┃     Columns     ┃   Status   ┃           │
│ ┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩           │
│ │ duplicate_document_… │ DuplicateColum… │      0.00s │    1 → 1     │       +1        │     ✓      │           │
│ │ question_generation… │ PromptBuilderB… │      0.00s │    1 → 1     │       +1        │     ✓      │           │
│ │ question_generation  │ LLMChatBlock    │      3.92s │    1 → 1     │       +1        │     ✓      │           │
│ │ extract_questions    │ LLMResponseExt… │      0.00s │    1 → 1     │       +1        │     ✓      │           │
│ │ parse_question_list  │ TagParserBlock  │      0.00s │    1 → 7     │       +1        │     ✓      │           │
│ │ answer_generation_p… │ PromptBuilderB… │      0.01s │    7 → 7     │       +1        │     ✓      │           │
│ │ answer_generation    │ LLMChatBlock    │      6.71s │    7 → 7     │       +1        │     ✓      │           │
│ │ extract_answer       │ LLMResponseExt… │      0.00s │    7 → 7     │       +1        │     ✓      │           │
│ │ parse_response_dict  │ TagParserBlock  │      0.00s │    7 → 7     │       +1        │     ✓      │           │
│ │ eval_faithful_prompt │ PromptBuilderB… │      0.01s │    7 → 7     │       +1        │     ✓      │           │
│ │ eval_faithful_llm_c… │ LLMChatBlock    │      5.43s │    7 → 7     │       +1        │     ✓      │           │
│ │ extract_eval_faithf… │ LLMResponseExt… │      0.00s │    7 → 7     │       +1        │     ✓      │           │
│ │ parse_eval_faithful  │ TagParserBlock  │      0.00s │    7 → 7     │       +2        │     ✓      │           │
│ │ eval_faithful_filter │ ColumnValueFil… │      0.00s │    7 → 6     │        —        │     ✓      │           │
│ ├──────────────────────┼─────────────────┼────────────┼──────────────┼─────────────────┼────────────┤           │
│ │ TOTAL                │ 14 blocks       │     16.11s │   6 final    │    21 final     │   14/14    │           │
│ └──────────────────────┴─────────────────┴────────────┴──────────────┴─────────────────┴────────────┘           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

           INFO     Flow 'Document Based Knowledge Tuning Dataset Generation Flow' completed       ]8;id=738250;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py\execution.py]8;;\:]8;id=420771;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/execution.py#548\548]8;;\
                    successfully: 6 final samples, 21 final columns                                                

Generation complete in 16.2s
Generated 6 QA pairs


Now compare.

In [49]:
print(f"Run 1 (TOC-heavy chunk): {len(result)} pairs")
print(f"Run 2 (rule-rich chunk): {len(result_v2)} pairs")
print()

for i in range(len(result_v2)):
    print(f"QA Pair #{i + 1}")
    print(f"  Question:     {result_v2['question'].iloc[i]}")
    print(f"  Answer:       {result_v2['response'].iloc[i]}")
    if 'faithfulness_judgment' in result_v2.columns:
        print(f"  Faithfulness: {result_v2['faithfulness_judgment'].iloc[i]}")
    print()

Run 1 (TOC-heavy chunk): 2 pairs
Run 2 (rule-rich chunk): 6 pairs

QA Pair #1
  Question:     What are the base ability percentages for Open Locks, Remove Traps, Pick Pockets, and Climb Walls for a Level 1 Thief?
  Answer:       Based on the provided document, the base ability percentages for a Level 1 Thief are as follows:

- **Open Locks:** 25%
- **Remove Traps:** 20%
- **Pick Pockets:** 30%
- **Climb Walls:** 80%
  Faithfulness: YES

QA Pair #2
  Question:     How do the Thief's ability percentages generally change as they progress from Level 1 to Level 19?
  Answer:       As the Thief progresses from Level 1 to Level 19, the ability percentages for their skills generally increase, reflecting their growing proficiency and expertise. Here's a summary of how each ability changes over these levels:

1. **Open Locks**: Starts at 25% at Level 1 and gradually increases to 87% by Level 19. The increase is steady, with slight acceleration as the levels progress.

2. **Remove Traps**: Begins

## 1.10 Export and Section Wrap-up 



In [51]:
df = result_v2.copy()
print(f"Full dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

keep_cols = ['question', 'response']
if 'faithfulness_judgment' in df.columns:
    keep_cols.append('faithfulness_judgment')
if 'relevancy_score' in df.columns:
    keep_cols.append('relevancy_score')

qa_df = df[keep_cols]
qa_df.to_csv("synthetic_qa_pairs.csv", index=False)
print(f"\nExported {len(qa_df)} pairs to synthetic_qa_pairs.csv")

Full dataset shape: (6, 21)
Columns: ['document', 'document_outline', 'domain', 'icl_document', 'icl_query_1', 'icl_query_2', 'icl_query_3', 'base_document', 'question_generation_prompt', 'question_list', 'extract_questions_content', 'question', 'answer_generation_prompt', 'response_dict', 'extract_answer_content', 'response', 'eval_faithful_prompt', 'eval_faithful_response_dict', 'extract_eval_faithful_content', 'faithfulness_explanation', 'faithfulness_judgment']

Exported 6 pairs to synthetic_qa_pairs.csv
